# 1. Configuración e ingesta inicial

In [30]:
import pandas as pd 
import os

data_path = os.path.join("..","data")

In [ ]:
# Carga de datos orders
orders = pd.read_csv(os.path.join(data_path,"olist_orders_dataset.csv"))
print(f"Dataset cargado con {len(orders)} filas.")
orders.head()

Dataset cargado con 99441 filas.


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [48]:
# Carga de datos customers
customers = pd.read_csv(os.path.join(data_path, "olist_customers_dataset.csv"))
print(f"Dataset cargado con {len(customers)} filas.")
customers.head()

Dataset cargado con 99441 filas.


,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


# 2. Inspección de calidad de datos:

In [32]:
print(orders.isnull().sum())

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64


# 3. Análisis de Inconsistencias (Hipótesis):

Datos **null** que llaman la atención

order_delivered_carrier_date     1783

order_delivered_customer_date    2965

**HIPOTESIS** 

Son pedidos cancelados ?

In [33]:
# Filtro solo pedidos que no llegaron al cliente
sin_entrega = orders[orders['order_delivered_customer_date'].isnull()]

# Que estados tienen estos pedidos
print("Estados de pedidos sin fecha de entrega:")
print(sin_entrega['order_status'].value_counts())

Estados de pedidos sin fecha de entrega:
order_status
shipped        1107
canceled        619
unavailable     609
invoiced        314
processing      301
delivered         8
created           5
approved          2
Name: count, dtype: int64


Ya que su mayor agrupación de nulls esta en shipped: 1107, nos indica que la hipotesis es correcta 

# 4. Transformación de Datos (Data Cleaning)

In [34]:
# Campos tiempo en formtato datetime 
columnas_fecha = [
    'order_purchase_timestamp', 'order_approved_at', 
    'order_delivered_carrier_date', 'order_delivered_customer_date', 
    'order_estimated_delivery_date'
]

for col in columnas_fecha:
    orders[col] = pd.to_datetime(orders[col])

print(orders.dtypes)

order_id                                    str
customer_id                                 str
order_status                                str
order_purchase_timestamp         datetime64[us]
order_approved_at                datetime64[us]
order_delivered_carrier_date     datetime64[us]
order_delivered_customer_date    datetime64[us]
order_estimated_delivery_date    datetime64[us]
dtype: object


Rango de fecha tiene sentido ?


In [35]:
print(f"Primer pedido: {orders['order_purchase_timestamp'].min()}")
print(f"Primer pedido: {orders['order_purchase_timestamp'].max()}")

Primer pedido: 2016-09-04 21:15:19
Primer pedido: 2018-10-17 17:30:18


# 5. Análisis de calidad de logística - Entregas tardias


In [36]:
# Filtro pedidos entregados 
entregados = orders[orders['order_status'] == 'delivered'].copy()

# diferencia de dias entre la entrega real y la estimada 
# número positivo, LLEGÓ TARDE
# negativo, LLEGÓ ANTES
entregados['delta_entrega'] = (entregados['order_delivered_customer_date'] - entregados['order_estimated_delivery_date']).dt.days

# clasificación de resultados
entregados['resultado_logistico'] = entregados['delta_entrega'].apply(lambda x: 'Tardío' if x > 0 else 'A tiempo/Anticipado')

# print resumen
print(entregados['resultado_logistico'].value_counts())

resultado_logistico
A tiempo/Anticipado    89944
Tardío                  6534
Name: count, dtype: int64


# 6. Preguntas que busco resolver con **PANDAS**

**A: El "Top de la Tardanza"**

Objetivo: Encuentra los 10 pedidos que más días de retraso tuvieron. Solo ver el order_id y los días de retraso


In [37]:
top_10_tardanza= entregados.sort_values(by='delta_entrega', ascending=False).head(10)
ranking_retrasos = top_10_tardanza[['order_id','delta_entrega']]
print(ranking_retrasos)

                               order_id  delta_entrega
55619  1b3190b2dfa9d789e1f14c05b647a14a          188.0
19590  ca07593549f1816d26a572e06dc1eab6          181.0
11399  47b40429ed8cce3aee9199792275433f          175.0
81401  2fe324febf907e3ea3f2aa9650869fa5          167.0
89130  285ab9426d6982034523a855f55a885e          166.0
61610  440d0d17af552815d15a9e41abe49359          165.0
68769  c27815f7e3dd0b926b58552628481575          162.0
38509  0f4519c5f1c541ddec9f21b3bddd533a          161.0
40847  d24e8541128cea179a11a65176e0a96f          161.0
54480  2d7561026d542c8dbd8f0daeadf67a43          159.0


**B: Porcentaje del total de pedidos que llegaron tarde**
Objetivo: Se sabe que 6534 llegaron tarde, cuanto representa en % del total


In [ ]:
# Total de registros
total_perdidos = len(entregados)
print(total_perdidos)

96478


In [42]:
# Total de registros tardios
total_tardios = entregados['resultado_logistico'].value_counts()
tardios = total_tardios['Tardío']
print(tardios)

6534


In [44]:
porcentaje_tardios = (tardios/total_perdidos)*100
print(f"El porcentaje de entrega tardía es: {porcentaje_tardios: .2f}%")

El porcentaje de entrega tardía es:  6.77%
